## Importing the Data


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# Download the dataset
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=21)

#print(train_labels)

# Normalize to [0, 1]
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

train_data = train_data.numpy()
test_data = test_data.numpy()

#train_labels = train_labels.reshape(train_labels.shape[0], 1)
#test_labels = test_labels.reshape(test_labels.shape[0], 1)


# Creating and Training the model

In [7]:
# Claim models
import keras
import matplotlib.pyplot as plt
import time
from keras.layers import Input
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Flatten, Dense
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model

# The input size/image
input_img = keras.Input(shape=(train_data.shape[1], 1))

#batches = [256, 128, 64, 32, 16, 8]

batchSize = 32
hiddenSize = 64
activationAlg="sigmoid"

data = []

for i in range(5):
  startTime = time.process_time()
  conv1 = Conv1D(8, 2, padding='same', activation=activationAlg)(input_img)
  conv1 = Conv1D(16, 3, padding='same', activation=activationAlg)(conv1)
  conv2 = Conv1D(8, 4, padding='same', activation=activationAlg)(input_img)
  conv2 = Conv1D(16, 2, padding='same', activation=activationAlg)(conv2)
  conv3 = MaxPooling1D(1, strides=3, padding='same')(input_img)
  conv3 = Conv1D(16, 2, padding='same', activation=activationAlg)(conv3)
  comb1 = keras.layers.concatenate([conv1, conv2, conv3], axis = 1)

  flat1 = Flatten()(comb1)
  norm1 = Dense(16, activation=activationAlg)(flat1)
  output = Dense(2, activation=activationAlg)(norm1)

  imModel = Model([input_img], output)

  #plot_model(imModel, to_file='imModel.png', show_shapes=True, show_layer_names=False)
  imModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Convert labels to categorical one-hot encoding
  one_hot_labels = keras.utils.to_categorical(train_labels, num_classes=2)

  progress = imModel.fit(train_data, one_hot_labels, epochs=100, batch_size=batchSize)
  losses = progress.history['loss']
  timePassed = time.process_time() - startTime
  y_pred_train=np.argmax(imModel.predict(train_data), axis=1)
  mse = mean_squared_error(train_labels,y_pred_train)
  #data.append(str(hiddenSize) + "," + str(mse) + "," + str(timePassed))

  for i in range(100):
    data.append(activationAlg + "," + str(i) + "," + str(losses[i]))

filename = "/content/drive/MyDrive/Colab Notebooks/EP-SM-IM.csv"
file = open(filename, "w")
#file.write("HiddenSize,Error,TrainingTime\n")
file.write("Activation,Epoch,Error\n")
for i in data:
  file.write(i + "\n")
file.close()


Epoch 1/100
125/125 [==============================] - 2s 9ms/step - loss: 0.7059 - accuracy: 0.5752
Epoch 2/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6751 - accuracy: 0.5952
Epoch 3/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6804 - accuracy: 0.5797
Epoch 4/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6809 - accuracy: 0.5800
Epoch 5/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6761 - accuracy: 0.5926
Epoch 6/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6761 - accuracy: 0.5925
Epoch 7/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6743 - accuracy: 0.5980
Epoch 8/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6759 - accuracy: 0.5932
Epoch 9/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6785 - accuracy: 0.5864
Epoch 10/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6745 - accur

# Testing the model

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# Overall F1 score
y_pred=np.argmax(model.predict(test_data), axis=1)
print(f1_score(test_labels, y_pred, average="macro"))
print(precision_score(test_labels, y_pred, average="macro"))
print(recall_score(test_labels, y_pred, average="macro"))
print(accuracy_score(test_labels, y_pred))

1  1.0
0.9837418861850742
0.984522316848773
0.9830357142857142
0.984
